# Einführung in ML Tutorial

In diesem Tutorial werden wir verschiedene Techniken des maschinellen Lernens auf Vibrationsdaten anwenden.
Unser Ziel ist es, den Zustand einer Maschine basierend auf Vibrationsdaten zu klassifizieren und spezifische Werte vorherzusagen.

**Inhalt:**
1. **Feature Engineering**: Vorbereitung der Daten.
2. **Supervised Learning (Klassifikation)**: Vorhersage von Kategorien (z. B. 'Normal' vs. 'Hoch').
3. **Supervised Learning (Regression)**: Vorhersage kontinuierlicher Werte.
4. **Unsupervised Learning**: Mustererkennung ohne Labels.
5. **Deep Learning Showcase**: Ein kurzer Einblick in neuronale Netze.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Daten laden
file_path = 'vibration_messung_2025-10-15_1704-1737.csv'
df = pd.read_csv(file_path)
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
print("Daten geladen:")
print(df.head())

## Feature Engineering

Rohdaten sind oft verrauscht. Wir wenden verschiedene Techniken an, um nützliche Informationen zu extrahieren:

1.  **Rolling Statistics**: Wir berechnen Mittelwert, Standardabweichung, Min und Max über ein Zeitfenster (Glättung).
2.  **Lag Features**: Wir verwenden Werte aus vorherigen Zeitschritten (z. B. t-1), da die Vergangenheit oft die Zukunft beeinflusst.
3.  **Rate of Change**: Wir berechnen die Differenz zwischen aufeinanderfolgenden Werten, um Trends zu erfassen.

Zusätzlich erstellen wir ein **Label** für das überwachte Lernen: Wenn der Mittelwert über 1000 liegt, klassifizieren wir dies als 'Hohe Vibration' (1), andernfalls als 'Normal' (0).

In [ ]:
# Berechne rollierende Statistiken (Fenstergröße 50)
window_size = 50
df_features = df[['wert']].rolling(window=window_size).agg(['mean', 'std', 'min', 'max']).dropna()
df_features.columns = df_features.columns.droplevel(0)

# --- NEU: Zusätzliche Features ---
# 1. Lag Features (Werte aus dem vorherigen Schritt)
df_features['mean_lag1'] = df_features['mean'].shift(1)
df_features['std_lag1'] = df_features['std'].shift(1)

# 2. Änderungsrate (Differenz zum vorherigen Schritt)
df_features['mean_change'] = df_features['mean'].diff()

# Entferne NaNs, die durch shift/diff entstanden sind
df_features.dropna(inplace=True)
# --------------------------------

# Erstelle Ziel-Label: "Hohe" Vibration (> 1000) vs "Normal"
# Wir verwenden den Mittelwert des Fensters für das Label
threshold = 1000
df_features['label'] = (df_features['mean'] > threshold).astype(int)

print("Features und Labels:")
print(df_features.head())
print(f"Verteilung der Labels:\n{df_features['label'].value_counts()}")

# --- VISUALISIERUNG ---
# 1. Rollierende Statistiken vs. Rohdaten (Erste 1000 Punkte zur Übersichtlichkeit)
plt.figure(figsize=(15, 5))
plt.plot(df['wert'].iloc[:1000], label='Rohdaten', alpha=0.5)
plt.plot(df_features['mean'].iloc[:1000], label='Rollierender Mittelwert (Fenster 50)', linewidth=2)
plt.title('1. Rollierende Statistiken: Glättung der Daten')
plt.legend()
plt.show()

# 2. Lag Features (Autokorrelation)
plt.figure(figsize=(6, 6))
plt.scatter(df_features['mean'], df_features['mean_lag1'], alpha=0.1, s=1)
plt.title('2. Lag Features: Aktueller Mittelwert vs. Vorheriger Mittelwert')
plt.xlabel('Mittelwert (t)')
plt.ylabel('Mittelwert (t-1)')
plt.show()

# 3. Änderungsrate
plt.figure(figsize=(15, 5))
plt.plot(df_features['mean_change'].iloc[:1000])
plt.title('3. Änderungsrate: Stabilität des Signals')
plt.ylabel('Änderung im Mittelwert')
plt.show()

## Supervised Learning: Klassifikation

Hier trainieren wir Modelle, um zu entscheiden, ob ein Zustand kritisch ist oder nicht.
Wir vergleichen verschiedene Algorithmen:
*   **Decision Tree (Entscheidungsbaum)**: Ein einfaches, interpretierbares Modell, das Daten basierend auf Feature-Werten aufteilt.
*   **Random Forest**: Ein Ensemble aus vielen Entscheidungsbäumen. Sehr robust.
*   **Support Vector Machine (SVM)**: Versucht, eine optimale Grenze zwischen Klassen zu finden.
*   **K-Nearest Neighbors (KNN)**: Klassifiziert basierend auf Ähnlichkeit zu Nachbarn.
*   **Gradient Boosting**: Baut Bäume sequenziell auf, um Fehler der vorherigen zu korrigieren.

In [ ]:
# Bibliotheken importieren
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Daten vorbereiten
X = df_features[['mean', 'std', 'min', 'max', 'mean_lag1', 'std_lag1', 'mean_change']]
y = df_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Dictionary zum Speichern der Ergebnisse für späteren Vergleich
results = {}

# --- 1. Decision Tree (Beispiel) ---
print("Trainiere Decision Tree...")
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)
results['Decision Tree'] = acc_dt
print(f"Decision Tree Genauigkeit: {acc_dt:.4f}")

# --- 2. Random Forest (Beispiel) ---
print("\nTrainiere Random Forest...")
rf_model = RandomForestClassifier(n_estimators=2, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
results['Random Forest'] = acc_rf
print(f"Random Forest Genauigkeit: {acc_rf:.4f}")

# --- 3. Support Vector Machine (TODO) ---
# TODO: Initialisiere SVC, trainiere das Modell, mache Vorhersagen und berechne die Genauigkeit
print("\nTrainiere SVM (TODO)...")
# svm_model = ...
# svm_model.fit(...)
# y_pred_svm = ...
# acc_svm = ...
# results['SVM'] = acc_svm
# print(f"SVM Genauigkeit: {acc_svm:.4f}")

# --- 4. K-Nearest Neighbors (TODO) ---
# TODO: Initialisiere KNeighborsClassifier, trainiere, mache Vorhersagen und berechne die Genauigkeit
print("\nTrainiere KNN (TODO)...")
# knn_model = ...
# ...

# --- 5. Gradient Boosting (TODO) ---
# TODO: Initialisiere GradientBoostingClassifier, trainiere, mache Vorhersagen und berechne die Genauigkeit
print("\nTrainiere Gradient Boosting (TODO)...")
# gb_model = ...
# ...

# --- Vergleich ---
if results:
    plt.figure(figsize=(10, 5))
    plt.bar(results.keys(), results.values(), color=['blue', 'orange', 'green', 'red', 'purple'])
    plt.title('Vergleich der Klassifikationsmodelle')
    plt.ylabel('Genauigkeit')
    plt.ylim(0, 1.1)
    plt.show()
else:
    print("Noch keine Modelle trainiert.")

## Supervised Learning: Regression

Bei der Regression wollen wir keinen Zustand (Klasse) vorhersagen, sondern einen **konkreten numerischen Wert**.
Aufgabe: Wir versuchen, den **Mittelwert** basierend auf den anderen Features ('std', 'min', 'max') vorherzusagen.

Modelle:
*   **Linear Regression**: Sucht nach einem linearen Zusammenhang.
*   **Decision Tree Regressor**: Ein Entscheidungsbaum für numerische Werte.
*   **Random Forest Regressor**: Ein Ensemble von Entscheidungsbäumen für kontinuierliche Werte.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np

# 1. Daten zuerst bereinigen
cols = ['std', 'min', 'max', 'mean', 'mean_lag1', 'std_lag1', 'mean_change']
df_clean = df_features[cols].replace([np.inf, -np.inf], np.nan).dropna()

X_reg = df_clean[['std', 'min', 'max', 'mean_lag1', 'std_lag1', 'mean_change']]
y_reg = df_clean['mean']

# 2. Aufteilen
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42, shuffle=False
)

print(f"Starte Training mit {len(X_train_r)} Beispielen...")

# --- 1. Decision Tree Regressor (Beispiel) ---
print("\nTrainiere Decision Tree Regressor...")
dt_reg = DecisionTreeRegressor(random_state=42, max_depth=10)
dt_reg.fit(X_train_r, y_train_r)
y_pred_dt = dt_reg.predict(X_test_r)

mse_dt = mean_squared_error(y_test_r, y_pred_dt)
r2_dt = r2_score(y_test_r, y_pred_dt)
print(f"Decision Tree - MSE: {mse_dt:.2f}, R2: {r2_dt:.4f}")

# --- 2. Linear Regression (TODO) ---
# TODO: Initialisiere LinearRegression, trainiere, mache Vorhersagen und berechne MSE/R2
print("\nTrainiere Linear Regression (TODO)...")
# lr_model = ...
# ...
# print(f"Linear Regression - MSE: {mse_lr:.2f}, R2: {r2_lr:.4f}")

# --- 3. Random Forest Regressor (TODO) ---
# TODO: Initialisiere RandomForestRegressor, trainiere, mache Vorhersagen und berechne MSE/R2
print("\nTrainiere Random Forest Regressor (TODO)...")
# rf_reg = ...
# ...
# print(f"Random Forest - MSE: {mse_rf:.2f}, R2: {r2_rf:.4f}")

## Unsupervised Learning: K-Means Clustering

Hier haben wir keine Labels. Der Algorithmus versucht selbstständig, Gruppen (Cluster) in den Daten zu finden.

In [ ]:
# Daten skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Trainiere K-Means (wir erwarten 2 Cluster: Normal und Hoch)
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Visualisierung
plt.figure(figsize=(15, 5))
plt.scatter(df_features.index, df_features['mean'], c=clusters, cmap='viridis', marker='.')
plt.title('K-Means Clustering der Vibrationsdaten')
plt.xlabel('Zeit')
plt.ylabel('Mittlere Vibration')
plt.show()

## Deep Learning Showcase

**Hinweis**: Für strukturierte Daten (Tabellen) wie diese sind klassische ML-Modelle (wie Random Forest oder Gradient Boosting) oft **besser, schneller und einfacher** zu trainieren als Deep Learning.
Deep Learning glänzt besonders bei unstrukturierten Daten wie Bildern, Audio oder komplexem Text.

Hier zeigen wir dennoch kurz, wie man ein einfaches neuronales Netz (MLP) mit Keras erstellt.

In [ ]:
# Daten skalieren (MinMax oft besser für NN)
scaler_nn = MinMaxScaler()
X_train_nn = scaler_nn.fit_transform(X_train)
X_test_nn = scaler_nn.transform(X_test)

# Modell erstellen
mlp_model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training
history_mlp = mlp_model.fit(X_train_nn, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# Evaluierung
loss, accuracy = mlp_model.evaluate(X_test_nn, y_test)
print(f"MLP Test Genauigkeit: {accuracy:.4f}")